<h1>
    <font color='green'>
                        Cálculos financeiros
     </font>
</h1>

Calcular os KPIs dos fundos.<br/>
1. [Informações](#1)<br>
a. [Versionamento](#2)<br>
b. [Funções usadas](#3)<br>
2. [Processo](#4)<br>
a. [Bibliotecas](#5) <br>
b. [Encontra arquivos que serão importados](#6)<br>
c. [Momento da importação](#7)<br>
d. [Tratamento de dados](#8)<br>
e. [Exclui dados desnecessários](#9)<br>
f. [Analise de um fundo](#10)<br>
g. [](#11)<br>

<a id =2></a><br>
### Versionamento

| Versão | Descrição | Data | Desenvolvedor
| :- |:------------- | :-: | :-:
|v1.0.0|Versão inicial|2021-06-13|Marco Cruz
|v1.0.1|Ajustes|2021-06-23|Marco Cruz


<a id =3></a><br>
### Funções

| Função | Descrição | Conceito
| :- |:------------- | :-:
|Beta| O Índice Beta é definido como a sensibilidade de um ativo <br/>em relação aos movimentos do mercado, ele é baseado no    <br/>conceito de risco sistemático, indicando a relação entre o <br/>retorno de um ativo e o retorno do mercado.  |<br/> $$ \beta_{pg} = \frac{\sigma_{pg,m}}{\sigma_{m}^2}$$ <br/>
| CAPM | CAPM (Capital Asset Pricing Model), que indica a expectativa de retorno <br/>de um ativo com base em uma série de variáveis — entre elas, o beta. <br/>Além do índice beta, o CAPM considera também a taxa de rendimento  <br/>de um ativo sem risco e a taxa de rentabilidade do mercado. |<br/> $$ \overline{r_{pg}} = r_f + \beta_{pg}(\overline{r_{m}} - r_f) $$ <br/>
| Sharpe | O Índice de Sharpe — também chamado de Sharpe Ratio — mensura <br/>a relação entre retorno e risco de um investimento. Por meio de seu <br/>resultado, você descobre se há vantagem entre determinado fundo <br/>de investimento na comparação com os ativos livres de risco. |<br/> $$ Sharpe = \frac{\overline{r_{pg}} - r_f}{\sigma_{pg}} $$ <br/>

<a id =5></a><br>
### Bibliotecas

In [1]:
#
# definição das bibliotecas necessárias
#
#import requests
#from bs4 import BeautifulSoup
import aux_config as cfg
import sqlite3 as sql
from pathlib import Path
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
#from datetime import datetime
import numpy as np
import aux_funcoes as fa
import aux_consultas_sql as csql

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 8)
pd.options.display.float_format = '{:,.4f}'.format


# ---------------------------------------------------------------------------
# How to disable Python warnings?
# https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings
#
# Muitas menssagens na etapa:
#                   Loop fundo à fundo para construir a variação da cota e PL
#
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
# ---------------------------------------------------------------------------


#print('-' * 75)
#print('-- definição das bibliotecas necessárias')
#print('-' * 75)

### Limpando atabelas de indicadores

In [2]:
conn = sql.connect(cfg.bancodedados)
with conn:
    cur = conn.cursor()
    cur.execute(csql.sqlDelSelicMes())
    cur.execute(csql.sqlDelRentabilidade())
    
#if conn:
#    conn.close()
fa.otimizarSQLite(conn)

<a id =6></a><br>
### Conectar ao banco de dados

In [3]:
#conn = sql.connect(cfg.bancodedados)

<a id =7></a><br>
### Gerar datasets do banco de dados

In [4]:
fundos = pd.read_sql(csql.sqlFundos(), conn).drop_duplicates() #makes sure we dont have duplicates
meses  = pd.read_sql(csql.sqlMeses(), conn).drop_duplicates()  #makes sure we dont have duplicates
cotas  = pd.read_sql(csql.sqlCotas(), conn)
selic  = pd.read_sql(csql.sqlSelic(), conn)
ibov   = pd.read_sql(csql.sqlIbov(), conn)



<a id =8></a><br>
### Gerar a lista de fundos a serem analisados

In [5]:
fundos

,CNPJ_FUNDO
0,00017024000153
1,00068305000135
2,00071477000168
3,00073041000108
4,00083181000167
...,...
26137,97711801000105
26138,97711806000120
26139,97711812000187
26140,97929197000180


In [6]:
fundos=fundos[fundos['CNPJ_FUNDO']=='00601692000123']
#fundos[fundos['CNPJ_FUNDO']=='00601692000123']

In [7]:
#fundos

<a id =8></a><br>
### Define janelas de análise da série temporal

In [8]:
meses

,dt_ult_pregao,ult_pregao,mes_01,mes_12,mes_24,mes_36,ano0,ano1,ano2,ano3
0,2021-06-28,2021-06-01,2021-05-01,2020-06-01,2019-06-01,2018-06-01,2021,2020,2019,2018


In [9]:
dt_ref=''.join(meses['dt_ult_pregao'].values)
mes00 =''.join(meses['ult_pregao'].values)
mes01 =''.join(meses['mes_01'].values)
mes12 =''.join(meses['mes_12'].values)
mes24 =''.join(meses['mes_24'].values)
mes36 =''.join(meses['mes_36'].values)
ano0  =''.join(meses['ano0'].values)
ano1  =''.join(meses['ano1'].values)
ano2  =''.join(meses['ano2'].values)
ano3  =''.join(meses['ano3'].values)


In [10]:
print(('Meses: '
       '\n - Últ. pregão: %(d00)s '
       '\n - Mês Últ. pr: %(m00)s '
       '\n - Mês fechado: %(m01)s '
       '\n - 1º mês M-12: %(m12)s '
       '\n - 1º mês M-24: %(m24)s '
       '\n - 1º mês M-36: %(m36)s ') % {'d00':dt_ref,'m00': mes00, 'm01': mes01, 'm12': mes12, 'm24':mes24, 'm36':mes36} )
      


Meses: 
 - Últ. pregão: 2021-06-28 
 - Mês Últ. pr: 2021-06-01 
 - Mês fechado: 2021-05-01 
 - 1º mês M-12: 2020-06-01 
 - 1º mês M-24: 2019-06-01 
 - 1º mês M-36: 2018-06-01 


<a id =9></a><br>
### Gera a base de cotações histórica

In [11]:
cotas

,CNPJ_FUNDO,ANO,MES,MES_COMPTC,VL_QUOTA,VL_PATRIM_LIQ
0,00017024000153,2016,12,2016-12-01,24.2545,"1,080,543.9300"
1,00017024000153,2017,01,2017-01-01,24.4657,"1,086,996.3900"
2,00017024000153,2017,02,2017-02-01,24.6224,"1,092,221.0100"
3,00017024000153,2017,03,2017-03-01,24.8637,"1,101,198.2600"
4,00017024000153,2017,04,2017-04-01,24.9736,"1,103,164.9400"
...,...,...,...,...,...,...
874836,97929213000134,2021,02,2021-02-01,9.3923,"71,375,464.8000"
874837,97929213000134,2021,03,2021-03-01,9.3823,"71,299,169.5500"
874838,97929213000134,2021,04,2021-04-01,9.5365,"72,470,912.7700"
874839,97929213000134,2021,05,2021-05-01,9.5911,"72,886,094.4800"


In [12]:
pd.options.display.float_format = '{:,.8f}'.format
selic

,MES_COTACAO,DT_COTACAO,ANO,MES,TAXA
0,2016-12-01,2016-12-01,2016,12,0.00050788
1,2016-12-01,2016-12-02,2016,12,0.00050788
2,2016-12-01,2016-12-05,2016,12,0.00050788
3,2016-12-01,2016-12-06,2016,12,0.00050788
4,2016-12-01,2016-12-07,2016,12,0.00050788
...,...,...,...,...,...
1143,2021-06-01,2021-06-24,2021,06,0.00016137
1144,2021-06-01,2021-06-25,2021,06,0.00016137
1145,2021-06-01,2021-06-28,2021,06,0.00016137
1146,2021-06-01,2021-06-29,2021,06,0.00016137


### Calcula a Selic mensal

In [27]:
meses_selic = selic.groupby(['MES_COTACAO']).agg({'DT_COTACAO':'count'})
meses_selic = meses_selic.reset_index()
meses_selic = meses_selic.drop(['DT_COTACAO'], axis=1)
meses_selic.rename(columns={'MES_COTACAO':'MES_SELIC'}, inplace=True)                            
#meses_selic

column_names = ['MES_SELIC', 'TX_SELIC']
rentSelic = pd.DataFrame(columns = column_names)

# Using for loop
i=0
for mes in meses_selic['MES_SELIC']:
    i+=1 
    #print(mes + ' - ' + str(i))
    rentSelictemp = fa.selic_mensal(selic[selic['MES_COTACAO']==mes], mes, '')
    #print(rentSelictemp)
    rentSelic=rentSelic.append(rentSelictemp)
    #rentSelic.to_sql('tbl_selic_mes', conn,  schema='investdb',index = False, if_exists='append')
    

In [28]:
rentSelic

,MES_SELIC,TX_SELIC
0,2016-12-01,0.01123315
0,2017-01-01,0.01086120
0,2017-02-01,0.00865084
0,2017-03-01,0.01052056
0,2017-04-01,0.00786581
...,...,...
0,2021-02-01,0.00134527
0,2021-03-01,0.00201080
0,2021-04-01,0.00207785
0,2021-05-01,0.00270326


,MES_SELIC,TX_SELIC


In [29]:
ibov

,MES_COTACAO,DT_COTACAO,ANO,MES,FECHAMENTO
0,2016-12-01,2016-12-29,2016,12,"60,227.00000000"
1,2017-01-01,2017-01-31,2017,01,"64,671.00000000"
2,2017-02-01,2017-02-24,2017,02,"66,662.00000000"
3,2017-03-01,2017-03-31,2017,03,"64,984.00000000"
4,2017-04-01,2017-04-28,2017,04,"65,403.00000000"
...,...,...,...,...,...
50,2021-02-01,2021-02-26,2021,02,"110,035.00000000"
51,2021-03-01,2021-03-31,2021,03,"116,634.00000000"
52,2021-04-01,2021-04-30,2021,04,"118,894.00000000"
53,2021-05-01,2021-05-31,2021,05,"126,216.00000000"


#### Loop fundo à fundo para construir a variação da cota e PL

In [30]:
# Using for loop
i=0
for cnpj in fundos['CNPJ_FUNDO']:
    try:
        i+=1 
        #print(cnpj + ' - ' + str(i))
        dados_fundo = cotas[cotas['CNPJ_FUNDO']==cnpj]

        #Trata horas, minutos e segundos do campo data/mês competência
        dados_fundo['MES_COMPTC']=dados_fundo['MES_COMPTC'].str.replace(' 00:00:00', '')

        dados_fundo['COTA_M1'] = dados_fundo['VL_QUOTA'].shift(1)
        dados_fundo['VAR_COTA'] = (dados_fundo['VL_QUOTA']/dados_fundo['COTA_M1'])-1

        dados_fundo['PL_M1']    = dados_fundo['VL_PATRIM_LIQ'].shift(1)    
        dados_fundo['VAR_PL']   = (dados_fundo['VL_PATRIM_LIQ']/dados_fundo['PL_M1'])-1

        dados_fundo['COTA_M1'] = dados_fundo['COTA_M1'].fillna(0)
        dados_fundo['PL_M1']   = dados_fundo['PL_M1'].fillna(0)
        dados_fundo['VAR_COTA'] = dados_fundo['VAR_COTA'].fillna(0)
        dados_fundo['VAR_PL']   = dados_fundo['VAR_PL'].fillna(0)

        # flags para calcular as posições de rentabilidade em 12, 24 e 36
        dados_fundo['M12']=(dados_fundo['MES_COMPTC']>=mes12) & (dados_fundo['MES_COMPTC']<=mes01)
        dados_fundo['M24']=(dados_fundo['MES_COMPTC']>=mes24) & (dados_fundo['MES_COMPTC']<=mes01)
        dados_fundo['M36']=(dados_fundo['MES_COMPTC']>=mes36) & (dados_fundo['MES_COMPTC']<=mes01)

        #dados_fundo
        retmes =  dados_fundo[['CNPJ_FUNDO','MES_COMPTC','VAR_COTA']]
        retmes.insert(2, 'JANELA', 'mensal')
        retmes.rename(columns={'MES_COMPTC':'DT_ULT_PREGAO', 'VAR_COTA':'RENTABILIDADE'}, inplace=True)
        
        result = pd.merge(retmes, rentSelic, how='left', on=['DT_ULT_PREGAO', 'MES_SELIC'])
        
        print(retmes)

        rent12 = fa.rentabilidade(dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['M12']==True)], dt_ref, '12 meses')
        rent24 = fa.rentabilidade(dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['M24']==True)], dt_ref, '24 meses')
        rent36 = fa.rentabilidade(dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['M36']==True)], dt_ref, '36 meses')
        renta0 = fa.rentabilidade(dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['ANO']==ano0) & (dados_fundo['MES_COMPTC']!=mes00)], dt_ref, ano0)
        renta1 = fa.rentabilidade(dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['ANO']==ano1) & (dados_fundo['MES_COMPTC']!=mes00)], dt_ref, ano1)
        renta2 = fa.rentabilidade(dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['ANO']==ano2) & (dados_fundo['MES_COMPTC']!=mes00)], dt_ref, ano2)
        renta3 = fa.rentabilidade(dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['ANO']==ano3) & (dados_fundo['MES_COMPTC']!=mes00)], dt_ref, ano3)
        frames = [retmes, rent12, rent24, rent36, renta0, renta1, renta2, renta3]
        rent =   pd.concat(frames)
        #rent.to_sql('tbl_rentabilidade', conn,  schema='investdb',index = False, if_exists='append')    
        print('-'*75)
    except:
        pass
    


<ipython-input-30-75c82b5ac614>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_fundo['MES_COMPTC']=dados_fundo['MES_COMPTC'].str.replace(' 00:00:00', '')
<ipython-input-30-75c82b5ac614>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_fundo['COTA_M1'] = dados_fundo['VL_QUOTA'].shift(1)
<ipython-input-30-75c82b5ac614>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [97]:
#
#retmes.info()
#retmes
#rentSelic.info()
#result = pd.merge(retmes, rentSelic, how='left', on=['DT_ULT_PREGAO', 'MES_SELIC'], validate="one_to_one")
#pd.merge(retmes, rentSelic, how='left', on=['DT_ULT_PREGAO', 'MES_SELIC']).dtypes
#new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
result = pd.merge(retmes,rentSelic, how="left", left_on=["DT_ULT_PREGAO"], right_on =["MES_SELIC"])
result = result[['CNPJ_FUNDO','DT_ULT_PREGAO','JANELA','RENTABILIDADE','TX_SELIC']]
result['DIF_SELIC'] = result['RENTABILIDADE'] - result['TX_SELIC']
result['AUX_VARIANC'] = result['DIF_SELIC'] **2
result['AUX_VARIANC_SORTINO'] = result[result['DIF_SELIC']<0]['AUX_VARIANC']
result['DT_ULT_PREGAO'] =  pd.to_datetime(result['DT_ULT_PREGAO'], infer_datetime_format=True)

# flags para calcular as posições de rentabilidade em 12, 24 e 36
result['M12']=(result['DT_ULT_PREGAO']>=mes12) & (result['DT_ULT_PREGAO']<=mes01)
result['M24']=(result['DT_ULT_PREGAO']>=mes24) & (result['DT_ULT_PREGAO']<=mes01)
result['M36']=(result['DT_ULT_PREGAO']>=mes36) & (result['DT_ULT_PREGAO']<=mes01)

#print(result.tail(13))
#print('-'*70)

janela =result[result['M12']==True]

#print(janela)
resulttot = janela.groupby(['CNPJ_FUNDO']).agg({'AUX_VARIANC':['count','sum'],'AUX_VARIANC_SORTINO':['count','sum']}).reset_index()
resulttot.columns = ["_".join(x) for x in resulttot.columns.ravel()]
resulttot['VARIANC']=resulttot['AUX_VARIANC_sum']/(resulttot['AUX_VARIANC_count']-1)
resulttot['RQ_VARIANC']=np.sqrt(resulttot['VARIANC'])
resulttot['DP'] = resulttot['RQ_VARIANC'] * np.sqrt(12)
resulttot




#meses_selic.rename(columns={'MES_COTACAO':'MES_SELIC'}, inplace=True)        

,CNPJ_FUNDO_,AUX_VARIANC_count,AUX_VARIANC_sum,AUX_VARIANC_SORTINO_count,AUX_VARIANC_SORTINO_sum,VARIANC,RQ_VARIANC,DP
0,00601692000123,12,0.03296686,3,0.00449990,0.00299699,0.05474474,0.18964136


In [96]:
np.sqrt(12)

3.4641016151377544

In [81]:
result

,CNPJ_FUNDO,DT_ULT_PREGAO,JANELA,RENTABILIDADE,TX_SELIC,DIF_SELIC,VARIANCIA,VARIANC_SORTINO,M12,M24,M36
0,00601692000123,2016-12-01,mensal,0.00000000,0.01123315,-0.01123315,0.00012618,0.00012618,False,False,False
1,00601692000123,2017-01-01,mensal,0.06979608,0.01086120,0.05893487,0.00347332,nan,False,False,False
2,00601692000123,2017-02-01,mensal,0.03502211,0.00865084,0.02637126,0.00069544,nan,False,False,False
3,00601692000123,2017-03-01,mensal,0.00886134,0.01052056,-0.00165922,0.00000275,0.00000275,False,False,False
4,00601692000123,2017-04-01,mensal,0.05219002,0.00786581,0.04432421,0.00196464,nan,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
50,00601692000123,2021-02-01,mensal,-0.05848509,0.00134527,-0.05983036,0.00357967,0.00357967,True,True,True
51,00601692000123,2021-03-01,mensal,0.03114248,0.00201080,0.02913169,0.00084866,nan,True,True,True
52,00601692000123,2021-04-01,mensal,0.01666821,0.00207785,0.01459036,0.00021288,nan,True,True,True
53,00601692000123,2021-05-01,mensal,0.04868792,0.00270326,0.04598465,0.00211459,nan,True,True,True


In [ ]:
conn.close()

In [ ]:
dados_fundo.tail(13)

In [ ]:
#rent.to_sql('tbl_rentabilidade', conn,  schema='investdb', index = False, if_exists='append')
#dados_fundo.tail(14)
rent

In [ ]:
rent.to_sql('tbl_rentabilidade', conn,  schema='investdb', index = False, if_exists='append')


In [ ]:
# dados_fundo.loc[1,'VAR_COTA']
#range(1, len(dados_fundo))
#range(0, (len(dados_fundo)))


In [ ]:
dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['ANO']==ano1) & (dados_fundo['MES_COMPTC']!=mes00)]

In [ ]:

rent

In [ ]:
#EXCLUI A PRIMEIRA LINHA
dados_fundo = dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0)]

#RECRIA INDICE
dados_fundo = dados_fundo.reset_index(drop=True)


dados_fundo.loc[0, 'RENTAB_COTA'] = (dados_fundo.loc[0,'VAR_COTA'] + 1) *1
for i in range(1, len(dados_fundo)):
    dados_fundo.loc[i, 'RENTAB_COTA'] = (dados_fundo.loc[i,'VAR_COTA'] + 1) * dados_fundo.loc[i-1, 'RENTAB_COTA']
    #print(i)

# PEGA O ULTIMO VALOR CALCULADO
rentab_cota = dados_fundo.loc[i, 'RENTAB_COTA']-1

dados_fundo

In [ ]:
#EXCLUI A PRIMEIRA LINHA
dados_fundo12 = dados_fundo[(dados_fundo['COTA_M1']!=0) & (dados_fundo['PL_M1']!=0) & (dados_fundo['M12']!=True)]

#RECRIA INDICE
dados_fundo12 = dados_fundo12.reset_index(drop=True)


dados_fundo12.loc[0, 'RENTAB_COTA'] = (dados_fundo12.loc[0,'VAR_COTA'] + 1) *1
for i in range(1, len(dados_fundo12)):
    dados_fundo12.loc[i, 'RENTAB_COTA'] = (dados_fundo12.loc[i,'VAR_COTA'] + 1) * dados_fundo12.loc[i-1, 'RENTAB_COTA']
    #print(i)

# PEGA O ULTIMO VALOR CALCULADO
rentab_cota12 = dados_fundo12.loc[i, 'RENTAB_COTA']-1
print(rentab_cota12)
print('-'*75)
print(dados_fundo12)

In [ ]:
# create excel writer object
writer = pd.ExcelWriter('output.xlsx')
# write dataframe to excel
dados_fundo.to_excel(writer)
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

In [ ]:
rentab_cota

In [ ]:
for i in range(len(dados_fundo)):
    print(dados_fundo.index[i] )
    ## Using loop

In [ ]:
fundo_filtro= df[df['CNPJ_FUNDO']=='00601692000123']

In [ ]:
query = '''
WITH
-- defines the filter table we will use in our query
max_data_pregao AS
(
SELECT MAX(DT_COMPTC) ultimoPregao FROM tbl_cotas WHERE VL_QUOTA<>0 and VL_PATRIM_LIQ > 100000
),
filter_query AS
(
SELECT DISTINCT C.CNPJ_FUNDO
  FROM tbl_cotas C
 INNER JOIN max_data_pregao D
    ON (C.DT_COMPTC = D.ultimoPregao)
 WHERE C.VL_PATRIM_LIQ >= 50000000
),
--defines the table with funds daily quotas
funds AS
(
SELECT C.DT_COMPTC as date, C.CNPJ_FUNDO as cnpj_fund, C.VL_QUOTA as fund_quota
  FROM tbl_cotas C 
 INNER JOIN max_data_pregao D
    ON(C.DT_COMPTC = D.ultimoPregao)
 INNER JOIN filter_query F
    ON(C.CNPJ_FUNDO = F.CNPJ_FUNDO) 
)
SELECT * from funds;
'''

In [ ]:
funds = pd.read_sql(query, conn).drop_duplicates() #makes sure we dont have duplicates

#fills any null values we had in our dataset with the following day value
cols = ['fund_quota']
funds[cols] = funds[cols].fillna(method = 'backfill')

#lets see how our dataset looks like
funds.head(5)

In [ ]:
fundo_filtro

In [ ]:
fundo_filtro['VAR_COTA']=(fundo_filtro['VL_QUOTA']/fundo_filtro['VL_QUOTA'].shift(1)-1)*100
fundo_filtro['VAR_COTA'].round(decimals=6)

In [ ]:
fundo_filtro['COTA_M1']=fundo_filtro['VL_QUOTA'].shift(1)

In [ ]:
fundo_filtro

In [ ]:
fundo_filtro

In [ ]:
# Initialise data to lists.
data = [{'cnpj': '1', 'janela': '12 meses', 'rentabilidade': 1.0247}]
  
# Creates DataFrame.
df = pd.DataFrame(data)
  
# Print the data
df